### Analysis: Will the Customer Accept the Coupon for Cheaper Restaurants?


In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import plotly.express as px

### Independent Investigation

Using the bar coupon example as motivation, you are to explore one of the other coupon groups and try to determine the characteristics of passengers who accept the coupons.  

Let's use coupon group *Cheaper Restaurant* group for our analysis.

In [2]:
# First create data frame for only those people where the chear restaurant, i.e. fast food places, coupons were offered.
data = pd.read_csv('data/coupons.csv')

# Drop car column - there is not much usable data available in that column
data.drop(columns='car', inplace=True)

# Create new dataframe where customers were offered cheaper restaurant 
# (known as fast food restaurant for this analysis going forward)
ff_coupons = data.query('coupon == "Restaurant(<20)"')

print(ff_coupons.shape)
print(ff_coupons.info())

(2786, 25)
<class 'pandas.core.frame.DataFrame'>
Index: 2786 entries, 0 to 12678
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   destination           2786 non-null   object
 1   passanger             2786 non-null   object
 2   weather               2786 non-null   object
 3   temperature           2786 non-null   int64 
 4   time                  2786 non-null   object
 5   coupon                2786 non-null   object
 6   expiration            2786 non-null   object
 7   gender                2786 non-null   object
 8   age                   2786 non-null   object
 9   maritalStatus         2786 non-null   object
 10  has_children          2786 non-null   int64 
 11  education             2786 non-null   object
 12  occupation            2786 non-null   object
 13  income                2786 non-null   object
 14  Bar                   2763 non-null   object
 15  CoffeeHouse           2739 non-

In [314]:
# let's convert some of the columns to numeric and add them as new column

#1 age
# current values: 21, 46, 26, 31, 41, 50plus, 36, below21 
# New Numeric Values: below21 -> 20, 21-25 -> 21, 26-30 -> 26, 31-35 -> 31
#                                36-40 -> 36, 41-45 -> 41, 46-50 -> 46, 50plus -> 51

def modify_age_value(x):
    if x == 'below21':
        return int(20)
    elif x == '50plus':
        return int(51)
    else:   
        return int(x)

ff_coupons['age_num'] = ff_coupons['age'].apply(modify_age_value)

print(ff_coupons['age_num'].describe())


count    2653.000000
mean       32.591406
std        10.474401
min        20.000000
25%        21.000000
50%        31.000000
75%        41.000000
max        51.000000
Name: age_num, dtype: float64


/var/folders/m8/xwy7k01n4rsbswwncxhbmrh00000gn/T/ipykernel_9783/3925237399.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ff_coupons['age_num'] = ff_coupons['age'].apply(modify_age_value)


In [316]:
#2 Income
# current values: income: $62500 - $74999, $12500 - $24999, $75000 - $87499, $50000 - $62499, $37500 - $49999, 
#                         $25000 - $37499, $100000 or More, $87500 - $99999, Less than $12500, 

# New Numeric Values: below21 -> 20, 21-25 -> 21, 26-30 -> 26, 31-35 -> 31
#                                36-40 -> 36, 41-45 -> 41, 46-50 -> 46, 50plus -> 51
import re
patt = r"^.[^\d]*(\d+)\s*"

def modify_income_value(x:str):

    if x == "Less than $12500":
        return int(12499)

    m = re.match(patt,x)
    if m:
        return int(m.group(1))



#ff_coupons['income'].apply(modify_income_value).unique()

ff_coupons['income_num'] = ff_coupons['income'].apply(modify_income_value)

print(ff_coupons['income_num'].describe())

count      2653.000000
mean      47865.537882
std       30622.232826
min       12499.000000
25%       25000.000000
50%       37500.000000
75%       75000.000000
max      100000.000000
Name: income_num, dtype: float64


/var/folders/m8/xwy7k01n4rsbswwncxhbmrh00000gn/T/ipykernel_9783/573209172.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ff_coupons['income_num'] = ff_coupons['income'].apply(modify_income_value)


In [317]:
#3 Y
#  Y: 1 -> "Accepted", 0-> "Rejected"

ff_coupons['Y_str'] = ff_coupons['Y'].map({0:"Rejected", 1:"Accepted"})

ff_coupons['Y_str'].unique()

/var/folders/m8/xwy7k01n4rsbswwncxhbmrh00000gn/T/ipykernel_9783/817769376.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ff_coupons['Y_str'] = ff_coupons['Y'].map({0:"Rejected", 1:"Accepted"})


array(['Accepted', 'Rejected'], dtype=object)

In [318]:
# Let's check if we have NaN values at this point
ff_coupons.isnull().sum()

destination             0
passanger               0
weather                 0
temperature             0
time                    0
coupon                  0
expiration              0
gender                  0
age                     0
maritalStatus           0
has_children            0
education               0
occupation              0
income                  0
Bar                     0
CoffeeHouse             0
CarryAway               0
RestaurantLessThan20    0
Restaurant20To50        0
toCoupon_GEQ5min        0
toCoupon_GEQ15min       0
toCoupon_GEQ25min       0
direction_same          0
direction_opp           0
Y                       0
age_num                 0
income_num              0
Y_str                   0
dtype: int64

In [319]:
#Now let's get all the unique values again.
rowinfo = ""
for col in ff_coupons.columns:
    rowinfo = col + ": "
    for val in sorted(ff_coupons[col].unique()):
        rowinfo += str(val) + ", "        
    print(rowinfo)

destination: Home, No Urgent Place, Work, 
passanger: Alone, Friend(s), Kid(s), Partner, 
weather: Rainy, Snowy, Sunny, 
temperature: 30, 55, 80, 
time: 10AM, 10PM, 2PM, 6PM, 7AM, 
coupon: Restaurant(<20), 
expiration: 1d, 2h, 
gender: Female, Male, 
age: 21, 26, 31, 36, 41, 46, 50plus, below21, 
maritalStatus: Divorced, Married partner, Single, Unmarried partner, Widowed, 
has_children: 0, 1, 
education: Associates degree, Bachelors degree, Graduate degree (Masters or Doctorate), High School Graduate, Some High School, Some college - no degree, 
occupation: Architecture & Engineering, Arts Design Entertainment Sports & Media, Building & Grounds Cleaning & Maintenance, Business & Financial, Community & Social Services, Computer & Mathematical, Construction & Extraction, Education&Training&Library, Farming Fishing & Forestry, Food Preparation & Serving Related, Healthcare Practitioners & Technical, Healthcare Support, Installation Maintenance & Repair, Legal, Life Physical Social Scienc

#### 1. Let's find acceptance rate for drivers based on their cheaper restaurant preference.

In [413]:
# Let's perform univariate analysis for RestaurantLessThan20 column
fig1_df = ff_coupons.groupby(['RestaurantLessThan20', 'Y_str'])[['Y']].count().rename(columns= {'Y': 'count'})

# We will create %val colunm for annotation
fig1_df['percent_val'] =  ((fig1_df / fig1_df.groupby(['RestaurantLessThan20'])[['count']].sum()) * 100).round(0).astype(int).astype(str) + '%'

# Reset index and sort values in decending order by count
fig1_df = fig1_df.reset_index().sort_values(by='count', ascending=False)

fig1 = px.bar(fig1_df, x="RestaurantLessThan20", y="count", color="Y_str", 
             text="percent_val", title="Coupon acceptance by Cheap Restaurant Preference")

fig1.show()


Acceptance rate is higher for fast food coupons. Drivers who normally don't go to fast food places also accepted more than 50% of time. 

#### 2. Univariate analysis by the age of driver.

In [419]:
fig2_df = ff_coupons.groupby(['age', 'Y_str'])[['Y']].count().rename(columns= {'Y': 'count'})

# We will create %val colunm for annotation
fig2_df['percent_val'] =  ((fig2_df / fig2_df.groupby(['age'])[['count']].sum()) * 100).round(0).astype(int).astype(str) + '%'

# Reset index and sort values in decending order by count
fig2_df = fig2_df.reset_index().sort_values(by='count', ascending=False)

fig2 = px.bar(fig2_df, x="age", y="count", color="Y_str", 
             text="percent_val", title="Coupon acceptance by Driver's age group")

fig2.show()

Acceptance rate is higher for younger drivers i.e. below Age of 35. 35 and above the acceptance rate is lower but still above 50%

#### 3. Let's check the rate of acceptance when drivers are relatively young (age 30 and below) for all income brackets 

In [ ]:
# age_num: 20, 21, 26, 31, 36, 41, 46, 51, 
# income_num: 12499, 12500, 25000, 37500, 50000, 62500, 75000, 87500, 100000, 
# Y_str: Accepted, Rejected,

age_lte_30 = 'age_num <= 30'

fig3_df = ff_coupons.query(age_lte_30).groupby(['income', 'Y_str'])[['Y']].count().rename(columns= {'Y': 'count'})

# We will create %val colunm for annotation
fig3_df['percent_val'] =  ((fig3_df / fig3_df.groupby(['income'])[['count']].sum()) * 100).round(0).astype(int).astype(str) + '%'

# Reset index 
fig3_df = fig3_df.reset_index().sort_values(by='count', ascending=False)

fig3 = px.bar(fig3_df, x="income", y="count", color="Y_str", 
             text="percent_val", title="Coupon acceptance by Income level for Age <= 30")

fig3.show()